# 模型参数的访问、初始化和共享
在[“线性回归的简洁实现”](../dlbasic/3.3linear-regression-gluon.ipynb) ⼀节中，我们通过init模块来初始化模型的全部参数。我们也介绍了
访问模型参数的简单⽅法。本节将深⼊讲解如何访问和初始化模型参数，以及如何在多个层之间
共享同⼀份模型参数。
我们先定义⼀个与上⼀节中相同的含单隐藏层的多层感知机。我们依然使⽤默认⽅式初始化它的
参数，并做⼀次前向计算。与之前不同的是，在这⾥我们从MXNet中导⼊了init模块，它包含
了多种模型初始化⽅法。

In [1]:
from mxnet import init, nd
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize() # 使⽤默认初始化⽅式
X = nd.random.uniform(shape=(2, 20))
Y = net(X) # 前向计算

## 访问模型参数
对于使⽤Sequential类构造的神经⽹络，我们可以通过⽅括号[]来访问⽹络的任⼀层。回忆⼀
下上⼀节中提到的Sequential类与Block类的继承关系。对于Sequential实例中含模型参数
的层，我们可以通过Block类的params属性来访问该层包含的所有参数。下⾯，访问多层感知
机net中隐藏层的所有参数。索引0表⽰隐藏层为Sequential实例最先添加的层。

In [4]:
net[0].params, type(net[0].params)

(dense0_ (
   Parameter dense0_weight (shape=(256, 20), dtype=float32)
   Parameter dense0_bias (shape=(256,), dtype=float32)
 ), mxnet.gluon.parameter.ParameterDict)

可以看到，我们得到了⼀个由参数名称映射到参数实例的字典（类型为ParameterDict类）。其中
权重参数的名称为dense0_weight，它由net[0]的名称（dense0_）和⾃⼰的变量名（weight）
组成。而且可以看到，该参数的形状为(256, 20)，且数据类型为32位浮点数（float32）。为了访
问特定参数，我们既可以通过名字来访问字典⾥的元素，也可以直接使⽤它的变量名。下⾯两种
⽅法是等价的，但通常后者的代码可读性更好

In [6]:
net[0].params['dense0_weight'], net[0].weight

(Parameter dense0_weight (shape=(256, 20), dtype=float32),
 Parameter dense0_weight (shape=(256, 20), dtype=float32))

Gluon⾥参数类型为Parameter类，它包含参数和梯度的数值，可以分别通过data函数和grad函
数来访问。因为我们随机初始化了权重，所以权重参数是⼀个由随机数组成的形状为(256,
20)的NDArray

In [8]:
net[0].weight.data()


[[ 0.06700657 -0.00369488  0.0418822  ... -0.05517294 -0.01194733
  -0.00369594]
 [-0.03296221 -0.04391347  0.03839272 ...  0.05636378  0.02545484
  -0.007007  ]
 [-0.0196689   0.01582889 -0.00881553 ...  0.01509629 -0.01908049
  -0.02449339]
 ...
 [ 0.00010955  0.0439323  -0.04911506 ...  0.06975312  0.0449558
  -0.03283203]
 [ 0.04106557  0.05671307 -0.00066976 ...  0.06387014 -0.01292654
   0.00974177]
 [ 0.00297424 -0.0281784  -0.06881659 ... -0.04047417  0.00457048
   0.05696651]]
<NDArray 256x20 @cpu(0)>

权重梯度的形状和权重的形状⼀样。因为我们还没有进⾏反向传播计算，所以梯度的值全为0。

In [10]:
net[0].weight.grad()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 256x20 @cpu(0)>

类似地，我们可以访问其他层的参数，如输出层的偏差值。

In [12]:
net[1].bias.data()


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 10 @cpu(0)>

最后，我们可以使⽤collect_params函数来获取net变量所有嵌套（例如通过add函数嵌套）
的层所包含的所有参数。它返回的同样是⼀个由参数名称到参数实例的字典。

In [13]:
net.collect_params()

sequential0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
  Parameter dense1_weight (shape=(10, 256), dtype=float32)
  Parameter dense1_bias (shape=(10,), dtype=float32)
)

这个函数可以通过正则表达式来匹配参数名，从而筛选需要的参数。

In [16]:
net.collect_params('.*weight')

sequential0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense1_weight (shape=(10, 256), dtype=float32)
)

## 初始化模型参数
我们在[“数值稳定性和模型初始化”](../dlbasic/3.15numerical-stability-and-init.ipynb) ⼀节中描述了模型的默认初始化⽅法：权重参数元素为[-
0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使⽤其他⽅法来初始化权
重。 MXNet的init模块⾥提供了多种预设的初始化⽅法。在下⾯的例⼦中，我们将权重参数初
始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [18]:
# ⾮⾸次对模型初始化需要指定force_reinit为真
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0]


[ 0.00139265 -0.00202259  0.01046625 -0.02317597  0.00228428 -0.0055537
 -0.00224368 -0.00034077 -0.00868913  0.00830154  0.00224619  0.00197733
 -0.00294902  0.00379219  0.00102535 -0.01005712 -0.01125984 -0.00325468
  0.00490046 -0.0105094 ]
<NDArray 20 @cpu(0)>

下⾯使⽤常数来初始化权重参数。

In [20]:
net.initialize(init=init.Constant(1), force_reinit=True)
net[0].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 20 @cpu(0)>

如果只想对某个特定参数进⾏初始化，我们可以调⽤Parameter类的initialize函数，它
与Block类提供的initialize函数的使⽤⽅法⼀致。下例中我们对隐藏层的权重使⽤Xavier随
机初始化⽅法。

In [21]:
net[0].weight.initialize(init = init.Xavier(), force_reinit = True)
net[0].weight.data()[0]


[-0.09281992  0.10184042 -0.02911698  0.01069678  0.12659115  0.05315876
 -0.11806712  0.03219472  0.13131225 -0.11840238  0.10895622 -0.1203045
 -0.01351677 -0.13093853 -0.05110312 -0.12192491 -0.07880946 -0.07750337
  0.0337538   0.10011691]
<NDArray 20 @cpu(0)>

## ⾃定义初始化⽅法
有时候我们需要的初始化⽅法并没有在init模块中提供。这时，可以实现⼀个Initializer类
的⼦类，从而能够像使⽤其他初始化⽅法那样使⽤它。通常，我们只需要实现_init_weight这
个函数，并将其传⼊的NDArray修改成初始化的结果。在下⾯的例⼦⾥，我们令权重有⼀半概率
初始化为0，有另⼀半概率初始化为[−10; −5]和[5; 10]两个区间⾥均匀分布的随机数。

In [22]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init',name,data.shape)
        data[:] = nd.random.uniform(low=-10,high=10,shape=data.shape)
        data *= data.abs()>=5
net.initialize(MyInit(),force_reinit=True)
net[0].weight.data()[0]

Init dense0_weight (256, 20)
Init dense1_weight (10, 256)



[ 0.        -0.        -5.0787363  0.         0.         8.537582
  0.         7.078802  -0.        -0.         0.        -0.
 -0.         6.451189   0.         0.        -0.        -0.
 -0.         9.378061 ]
<NDArray 20 @cpu(0)>

此外，我们还可以通过Parameter类的set_data函数来直接改写模型参数。例如，在下例中我
们将隐藏层参数在现有的基础上加1。

In [23]:
net[0].weight.set_data(net[0].weight.data()+1)
net[0].weight.data()[0]


[ 1.         1.        -4.0787363  1.         1.         9.537582
  1.         8.078802   1.         1.         1.         1.
  1.         7.451189   1.         1.         1.         1.
  1.        10.378061 ]
<NDArray 20 @cpu(0)>

## 共享模型参数
在有些情况下，我们希望在多个层之间共享模型参数。 [“模型构造”](4.1model_construction.ipynb) ⼀节介绍了如何在Block类
的forward函数⾥多次调⽤同⼀个层来计算。这⾥再介绍另外⼀种⽅法，它在构造层的时候指定
使⽤特定的参数。如果不同层使⽤同⼀份参数，那么它们在前向计算和反向传播时都会共享相同
的参数。在下⾯的例⼦⾥，我们让模型的第⼆隐藏层（shared变量）和第三隐藏层共享模型参数。

In [24]:
net = nn.Sequential()
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()
X = nd.random.uniform(shape=(2, 20))
net(X)
net[1].weight.data()[0] == net[2].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 8 @cpu(0)>

我们在构造第三隐藏层时通过params来指定它使⽤第⼆隐藏层的参数。因为模型参数⾥包含了
梯度，所以在反向传播计算时，第⼆隐藏层和第三隐藏层的梯度都会被累加在shared.params.grad()⾥。
## 小结
- 有多种⽅法来访问、初始化和共享模型参数。
- 可以⾃定义初始化⽅法。